# ACOPF with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl) using [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports non-linear AC optimal power flow through a deep integration
with [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl). This example shows a
single multi-period optimization of economic dispatch with a full representation of
AC optimal power flow.

## Dependencies
We can use the same RTS data and some of the initialization as in
[OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSimulations_examples", "1_operations_problems.jl"));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/

Since we'll be doing non-linear optimization, we need a solver that supports non-linear
problems. Ipopt is quite good.

In [2]:
using Ipopt
solver = optimizer_with_attributes(Ipopt.Optimizer)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[])

In the [OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
we defined a unit-commitment problem with a copper plate representation of the network.
Here, we want do define an economic dispatch (linear generation decisions) with an ACOPF
network representation.
So, starting with the network, we can select from _almost_ any of the endpoints on this
tree:

In [3]:
TypeTree(PSI.PM.AbstractPowerModel,  init_expand = 10, scopesep="\n")

PowerModels
AbstractPowerModel
├──PowerModels
│  AbstractACRModel
│  ├──ACRPowerModel
│  └──PowerModels
│     AbstractIVRModel
│     └──PowerModels
│        IVRPowerModel (0 children)
├──PowerModels
│  AbstractACPModel
│  └──ACPPowerModel
├──PowerModels
│  AbstractConicModel
│  ├──PowerModels
│  │  AbstractWRConicModel
│  │  └──PowerModels
│  │     AbstractSOCWRConicModel (1 children)
│  └──PowerModels
│     AbstractWRMModel
│     └──PowerModels
│        AbstractSDPWRMModel (2 children)
├──PowerModels
│  AbstractWRModel
│  ├──PowerModels
│  │  AbstractQCWRModel
│  │  ├──PowerModels
│  │  │  AbstractQCRMPowerModel (1 children)
│  │  └──PowerModels
│  │     AbstractQCLSModel (1 children)
│  └──PowerModels
│     AbstractSOCWRModel
│     └──SOCWRPowerModel (0 children)
├──PowerModels
│  AbstractLPACModel
│  └──PowerModels
│     AbstractLPACCModel
│     └──LPACCPowerModel (0 children)
├──PowerModels
│  AbstractBFModel
│  ├──PowerModels
│  │  AbstractBFAModel
│  │  └──PowerModels
│  │     BFAPowerModel (0 children)
│  ├──PowerModels
│  │  AbstractBFConicModel
│  │  └──PowerModels
│  │     AbstractSOCBFConicModel (1 children)
│  └──PowerModels
│     AbstractBFQPModel
│     └──PowerModels
│        AbstractSOCBFModel (1 children)
├──PowerModels
│  AbstractActivePowerModel
│  ├──PowerModels
│  │  AbstractDCPModel
│  │  ├──PowerModels
│  │  │  AbstractDCPLLModel (1 children)
│  │  ├──StandardPTDFModel (0 children)
│  │  ├──PowerModels
│  │  │  AbstractNFAModel (1 children)
│  │  ├──PowerModels
│  │  │  AbstractDCMPPModel (1 children)
│  │  └──DCPPowerModel (0 children)
│  ├──CopperPlatePowerModel
│  └──AreaBalancePowerModel
└──PowerModels
   AbstractACTModel
   └──ACTPowerModel

For now, let's just choose a standard ACOPF formulation.

In [4]:
ed_template = template_economic_dispatch(network = ACPPowerModel)


Operations Problem Specification

  transmission:  ACPPowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalRampLimited
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchReservoirBudget
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        d

Currently  energy budget data isn't stored in the RTS-GMLC dataset.

In [5]:
ed_template.devices[:Hydro]= DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver)

DeviceModel{HydroEnergyReservoir,HydroDispatchRunOfRiver}(HydroEnergyReservoir, HydroDispatchRunOfRiver, nothing, ServiceModel[])

Now we can build a 4-hour economic dispatch / ACOPF problem with the RTS data.

In [6]:
problem = OperationsProblem(
    EconomicDispatchProblem,
    ed_template,
    sys,
    horizon = 4,
    optimizer = solver,
    balance_slack_variables = true,
)

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/OXke7/src/base.jl:278
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/i0Xhh/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/i0Xhh/src/devices_models/devices/thermal_generation.jl:595
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/i0Xhh/src/devices_models/device_constructors/common/constructor_validations.jl:3



Operations Problem Specification

  transmission:  ACPPowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalRampLimited
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device

And solve it ...

In [7]:
solve!(problem)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    15152
Number of nonzeros in inequality constraint Jacobian.:     5420
Number of nonzeros in Lagrangian Hessian.............:    21120

Total number of variables............................:     5832
                     variables with only lower bounds:     1168
                variables with lower and upper bounds:     3292
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2804
Total number of inequality constraints...............:     3644
        inequality constraints with only lower bounds:     1244
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2400

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Results
========

Variables
=========

P__ThermalStandard: (4, 76)
P__RenewableDispatch: (4, 30)
γ⁺__Q: (4, 73)
FqFT__TapTransformer: (4, 15)
FpTF__Line: (4, 105)
Q__HydroEnergyReservoir: (4, 19)
Q__RenewableDispatch: (4, 30)
P__HydroEnergyReservoir: (4, 19)
γ⁺__P: (4, 73)
FqTF__HVDCLine: (4, 1)
FpFT__Line: (4, 105)
FpFT__HVDCLine: (4, 1)
Q__ThermalStandard: (4, 76)
FqTF__Line: (4, 105)
FqTF__TapTransformer: (4, 15)
Vm__Bus: (4, 73)
FqFT__Line: (4, 105)
theta__Bus: (4, 73)
FpTF__TapTransformer: (4, 15)
γ⁻__P: (4, 73)
FpFT__TapTransformer: (4, 15)
FqFT__HVDCLine: (4, 1)
FpTF__HVDCLine: (4, 1)
γ⁻__Q: (4, 73)


Optimizer Log
-------------
        timed_solve_time = 5.761824291
        solve_bytes_alloc = 95910568
        solve_time = 5.719479084014893
        obj_value = 2.0107518781428106e7
        solver = Ipopt
        sec_in_gc = 0.0
        dual_status = FEASIBLE_POINT
        primal_status = FEASIBLE_POINT
        termination_status = LOCALLY_SOLVED


Total Cost: OBJECTIVE_FUNCTION = 2.0107518781428106e7

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*